In [41]:
import pandas as pd
import numpy as np
import time
from subprocess import Popen
from multiprocessing import Process, Queue
import os

 
start_time = time.time()

def FormatFile(initformat,finalformat,datainit,datafinish,nameofbash):
    Popen(nameofbash+' %s %s %s %s' % (str(initformat),str(finalformat),str(datainit),str(datafinish)), shell=True)

def ReadFile(nameoffile,nrowss,skiprowss):
    #Popen(nameofbash+' %s %s %s %s' % (str(initformat),str(finalformat),str(datainit),str(datafinish)), shell=True)
    #time.sleep(1)
    while True:
        try:
            df=pd.read_table(nameoffile,nrows=nrowss,skiprows=skiprowss)
            break
        except:
            continue
    df.columns=['id','first_name','last_name','account_number','email']
    df.fillna('',inplace=True)
    return df
    
    
def numbers(number):
    var=''
    for s in number.split():
        if s.isdigit():
            var=s       
    return var

def isstring(number):
    var=''
    for s in number.split():
        if s.isalpha():
            var=s       
    return var

def isemail(number):
    var=''
    for s in number.split():
        if '@' in s:
            var=s       
    return var

def IdNotNull (NameofFile,sepa,NameofOut):
    
    final=pd.read_csv(NameofFile,sep=sepa)
    x=list(range(1,final.shape[0]+1))
    ids = pd.DataFrame({'Newid':x})
    final['id']=ids['Newid']
    
    final.to_csv(NameofOut,index=False, sep=sepa)
    
def cleandata(df,nameexport):
    


    #check id
    df['id_ok']=df['id'].astype(str).apply(numbers)
    df['id_name']=df['id'].astype(str).apply(isstring)
    df['id_email']=df['id'].astype(str).apply(isemail)

    #check first_name
    df['first_name_ok']=df['first_name'].astype(str).apply(isstring)
    df['first_name_account']=df['first_name'].astype(str).apply(numbers)
    df['first_name_email']=df['first_name'].astype(str).apply(isemail)

    #check last_name
    df['last_name_ok']=df['last_name'].astype(str).apply(isstring)
    df['last_name_account']=df['last_name'].astype(str).apply(numbers)
    df['last_name_email']=df['last_name'].astype(str).apply(isemail)

    #check account_number
    df['account_number_ok']=df['account_number'].astype(str).apply(numbers)
    df['account_number_name']=df['account_number'].astype(str).apply(isstring)
    df['account_number_email']=df['account_number'].astype(str).apply(isemail)

    #check email
    df['email_ok']=df['email'].astype(str).apply(isemail)
    df['email_name']=df['email'].astype(str).apply(isstring)
    df['email_number']=df['email'].astype(str).apply(numbers)


    #Clean bad positions
    df['id'] = np.where(df['id_ok']!='', df['id_ok'], '')
    df['first_name'] = np.where(df['first_name_ok']!='', df['first_name_ok'], '')
    df['last_name'] = np.where(df['last_name_ok']!='', df['last_name_ok'], '')
    df['account_number'] = np.where(df['account_number_ok']!='', df['account_number_ok'], '')
    df['email'] = np.where(df['email_ok']!='', df['email_ok'], '')

    #IDs
    df['first_name'] = np.where(df['id_name']!='', df['id_name'],df['first_name_ok'])
    df['first_name_ok'] = df['first_name']
    df['email'] = np.where(df['id_email']!='', df['id_email'],df['email_ok'])
    df['email_ok']= df['email']

    #FirstNames
    df['account_number'] = np.where(df['first_name_account']!='', df['first_name_account'],df['account_number_ok'])
    df['account_number_ok']=df['account_number']
    df['email'] = np.where(df['first_name_email']!='', df['first_name_email'],df['email_ok'])
    df['email_ok']= df['email']

    #LastNames
    df['account_number'] = np.where(df['last_name_account']!='', df['last_name_account'],df['account_number_ok'])
    df['account_number_ok']=df['account_number']
    df['email'] = np.where(df['last_name_email']!='', df['last_name_email'],df['email_ok'])
    df['email_ok']= df['email']

    #AccountsNumbers
    df['first_name'] = np.where(df['account_number_name']!='', df['account_number_name'],df['first_name_ok'])
    df['first_name_ok'] = df['first_name']
    df['email'] = np.where(df['account_number_email']!='', df['account_number_email'],df['email_ok'])
    df['email_ok'] = df['email_name']

    #Emails
    df['first_name'] = np.where(df['email_name']!='', df['email_name'],df['first_name_ok'])
    df['first_name_ok'] = df['first_name']
    df['account_number'] = np.where(df['email_number']!='', df['email_number'],df['account_number_ok'])
    df['account_number_ok'] = df['account_number']
    #print(df)
    #FinalID
    #x=list(range(1,df.shape[0]+1))
    #df_2 = pd.DataFrame({'Newid':x})
    #df['id']=df_2['Newid']


    col=['id','first_name','last_name','account_number','email']
    df=df[col]
    
    
    
    df.to_csv(nameexport,index=False)
    return(df)
    
def main():

    start_time = time.time()
    
    FormatFile('UTF-16LE','UTF-8','data.tsv','./data-utf8.tsv','./format.sh')
    df_1=ReadFile('./data-utf8.tsv',20,None)
    df_2=ReadFile('./data-utf8.tsv',None,20)
   
 
    q = Queue()
    process_one = Process(target=cleandata, args=(df_1,'df_1.csv'))
    process_two = Process(target=cleandata, args=(df_2,'df_2.csv'))
    
    process_one.start()
    process_two.start()
    
    q.close()
    q.join_thread()
    process_one.join()
    process_two.join()
  

    df_3=pd.read_csv('df_1.csv',usecols=['id','first_name','last_name','account_number','email'])
    df_4=pd.read_csv('df_2.csv',usecols=['id','first_name','last_name','account_number','email'])
    
    frames=[df_3,df_4]
    result = pd.concat(frames)
    result.to_csv('FinalData(id_old).tsv',index=False, sep='\t')
    os.remove('df_1.csv')
    os.remove('df_2.csv')
    
    IdNotNull('FinalData(id_old).tsv','\t','FinalData(id_new).tsv')
    
    print("--- Data processed in %s seconds ---" % (time.time() - start_time))
if __name__ == "__main__":
    main()


--- Data processed in 0.0537867546081543 seconds ---
